In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import json as jsc
import re
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import hamming_loss
import csv
nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt')


from gensim.models import KeyedVectors
# Load vectors directly from the file
model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/IR Project/GoogleNews-vectors-negative300.bin.gz', binary=True)

Id = []
Name = []
Description = []
Genre = []

l = []

with open("/content/drive/MyDrive/IR Project/booksummaries.txt", 'r', encoding="utf8") as file:
    r = csv.reader(file, dialect='excel-tab')
    for row in r:
        l.append(row)

for i in l:
    Id.append(i[0])
    Name.append(i[2])
    Genre.append(i[5])
    Description.append(i[6])

df = pd.DataFrame({'id': Id, 'name': Name, 'description': Description,
                       'genre': Genre})

df.head()
df.tail()

df.dtypes

df['id'] = df['id'].astype('int')

df.drop(df[df['genre']==''].index, inplace=True)
df[df['genre']=='']

df.tail()

print(list(np.where(df['name']=='The Chinese Gold Murders')))
df.iloc[7240]

jsc.loads(df['genre'][0]).values()

l=[]
for i in df['genre']:
    l.append(list(jsc.loads(i).values()))

df['u-genre'] = l

setOfgenre = sum(l,[])
setOfgenre = set(setOfgenre)

print(len(setOfgenre))

def updateDescription(s):
    s = re.sub("\'", "", s)
    s = re.sub("[^a-zA-Z]"," ",s)
    s = ' '.join(s.split())
    s = s.lower()
    return s

df['u-description'] = df['description'].apply(lambda x: updateDescription(x))

df.head()

df=df.drop(['genre', 'description'], axis=1)

df.head()

df.rename(columns = {'u-genre':'genre'}, inplace = True)

df.rename(columns = {'u-description':'description'}, inplace = True)

df.head()

df.dtypes

df.info()

df.shape

df.describe()

def preprocess_SW(desc):
    setSW = set(stopwords.words('english'))
    tokens = word_tokenize(desc)
    newDesc = [x for x in tokens if not x.lower() in setSW]
    return ' '.join(newDesc)

preprocess_SW("old major the old boar on the manor farm calls the animals on the farm for a meeting where he compares the humans to parasites and teaches the animals a revolutionary song beasts of england when major dies two young pigs snowball and napoleon assume command and turn his dream into a philosophy")    

df['description w/o SW'] = df['description'].apply(lambda x: preprocess_SW(x))

df.head()

print(setOfgenre)

book_genres = setOfgenre
len(book_genres)

def preprocess_genres(i):
    t = stopwords.words("english")
    pattern = re.compile(r'\b(' + r'|'.join(t) + r')\b\s*')
    tokens = nltk.word_tokenize(i)
    s = ""
    s = ", ".join(tokens)
    stopword_removed =[]    
    for word in tokens:
      word = pattern.sub('', word)
      if(len(word)> 0):
        stopword_removed.append(word)
    final_genre = ''  
    for word in stopword_removed:
      final_genre += (word+' ')  
    final_genre = final_genre.strip()  
    return final_genre

temp_genre = []
for i in df['genre']:
  temp_genre2 = []
  for j in i:
    temp_genre2.append(preprocess_genres(j))
  temp_genre.append(temp_genre2)
df['m-genre'] = temp_genre
print(df)


temp_genre = []
for i in df['m-genre']:
  temp_genre2 = i
  for j in i:
    if (j ==  'Bangsian fantasy'):
      temp_genre2.remove('Bangsian fantasy')   #related to afterlife book
      temp_genre2.append('Afterlife fantasy')
    elif (j ==  'Biopunk'):
      temp_genre2.remove('Biopunk')
      temp_genre2.append('Biotechnology fiction')  #related to biotech fiction
    elif (j ==  'Edisonade'):
      temp_genre2.remove('Edisonade')  #related to great inventor, engineer
      temp_genre2.append('Inventor fiction')  
    elif (j == 'Elizabethan romance'):
      temp_genre2.remove('Elizabethan romance')
    elif (j == 'Ergodic literature'):
      temp_genre2.remove('Ergodic literature')
    elif (j == 'Humour'):
      temp_genre2.remove('Humour')     
      temp_genre2.append('Humor')
    elif (j == 'Künstlerroman'):
      temp_genre2.remove('Künstlerroman') 
    elif (j == 'Postcyberpunk'):
      temp_genre2.remove('Postcyberpunk')     
      temp_genre2.append('cyberpunk')  
    elif (j == 'Robinsonade'):
      temp_genre2.remove('Robinsonade')
  temp_genre.append(temp_genre2)
df['m-genre'] = temp_genre
print(df)

setOfgenre = sum(df['m-genre'],[])
setOfgenre = set(setOfgenre)

print(len(setOfgenre))

bk = list(setOfgenre)

genre_vectors = []

#main_genres = ['Fiction','Comedy','Adventure', 'Non-fiction', 'Romance', 'Mystery', 'Science Fiction', 'Fantasy', 'Horror', 'Historical Fiction', 'Young Adult', 'Spirituality']
main_genres = ['History', 'Poetry', 'Autobiography', 'Romance','Biography', 'Literary', 'Spirituality', 'Western fiction', 'Comedy', 'Adventure',
               'Dark', 'Sociology', 'Comic', 'Anthology', 'Drama', 'Thriller', 'Action',  'Conspiracy', 'Catastrophic', 
               'Young adult','Fantasy', 'Travel', 'Science', 'Speculative', 'Horror', 'Mystery', 'Non fiction']
for genre in bk:
    genre = genre.replace('-', ' ')
    genre = genre.replace('\'', ' ')
    #genre = genre.replace('edisonade', 'inventor')
    tokens = genre.lower().split()
    vector = sum(model[token] for token in tokens) / len(tokens)
    genre_vectors.append(vector)


main_genre_vectors = []
for genre in main_genres:
    genre = genre.replace('-', ' ')
    tokens = genre.lower().split()
    vector = sum(model[token] for token in tokens) / len(tokens)
    main_genre_vectors.append(vector)
    
book_genre_df = pd.DataFrame()
book_genre_df['genre'] = bk    

from sklearn.metrics.pairwise import cosine_similarity
u_genre_list = []
for j in range(0,223):
  similarity_list = []
  map_j_list = []
  for i in range(len(main_genre_vectors)):
    similarity = cosine_similarity([genre_vectors[j]], [main_genre_vectors[i]])[0][0]
    similarity_list.append(similarity)
    map_j_list.append(i)
  idx = similarity_list.index(sorted(similarity_list)[-1])
  u_genre_list.append(main_genres[idx])

book_genre_df['m-genre'] = u_genre_list

book_genre_df

temp_genre = []
for i in df['m-genre']:
  temp_genre2 = []
  for j in i:
    mapped_genre = book_genre_df[book_genre_df['genre']==j]['m-genre'].values[0]
    temp_genre2.append(mapped_genre)
  temp_genre.append(temp_genre2)
df['m-genre'] = temp_genre
print(df)

mBin = MultiLabelBinarizer()
mBin.fit(df['m-genre'])
label = mBin.transform(df['m-genre'])

##Splitting into training and testing data
trainData, testData, trainLabel, testLabel = train_test_split(df['description w/o SW'],label, random_state=0, train_size = .80)

##Converting the input text to feature vectors
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)
trainData_v = tfidf_vectorizer.fit_transform(trainData)
testData_v = tfidf_vectorizer.transform(testData)

lr = LogisticRegression()
clf = OneVsRestClassifier(lr)
clf.fit(trainData_v, trainLabel)

y_pred = clf.predict(testData_v)
f1_score(testLabel, y_pred, average="micro"), accuracy_score(testLabel, y_pred)

def predict(m):
    m = updateDescription(m)
    m = preprocess_SW(m)
    m_vec = tfidf_vectorizer.transform([m])
    m_pred = clf.predict(m_vec)
    return mBin.inverse_transform(m_pred)

test_df = pd.DataFrame(testData)
test_df.head()

multilabel_confusion_matrix(testLabel, y_pred)

hamming_loss(testLabel,y_pred)

c = 0
for i in predict(test_df['description w/o SW'][12457]):
  j = set(i)
  for m in range(len(df['m-genre'])):
    n = set(df['m-genre'].iloc[m])
    if len(n.intersection(j)) == len(j):
      if len(n.intersection(j)) == len(n):
        print(df['name'].iloc[m])
        c+=1

print(c)

for i in test_df.index[0:15]:
  print(i)
  print("Predicted: ", predict(test_df['description w/o SW'][i]))
  print("Actual: ", df['m-genre'][i])
  print("Name: ", df['name'][i])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[array([7240])]
227
<class 'pandas.core.frame.DataFrame'>
Int64Index: 12841 entries, 0 to 16558
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           12841 non-null  int64 
 1   name         12841 non-null  object
 2   genre        12841 non-null  object
 3   description  12841 non-null  object
dtypes: int64(1), object(3)
memory usage: 1017.6+ KB
{'Adventure', "Boys' school stories", 'Politics', 'Low fantasy', 'Inspirational', 'Dark fantasy', 'Historical romance', 'Cozy', 'Education', 'Postcyberpunk', 'Polemic', 'Vampire fiction', 'Gamebook', 'Business', 'School story', 'Anthropology', 'Subterranean fiction', 'Young adult literature', 'Ghost story', 'Apocalyptic and post-apocalyptic fiction', 'Absurdist fiction', 'Human extinction', 'Spirituality', 'Science fantasy', 'Youth', 'Creative nonfiction', 'Crime Fiction', 'Chivalric romance', 'Juvenile fantasy', 'Mystery', 'Utopian and dystopian fiction', 'Time trav

In [23]:
test_df

,description w/o SW
12457,incident saudi arabia touches world war iii un...
1403,british risen power egypt suppress worship old...
6325,year old annabel hope jayquith beautiful famou...
11366,midsomer worthy writers circle group amateur n...
4374,book follows jason worthing also known jazz bo...
...,...
4976,stephanie plum back wants revenge time shes tr...
4654,moose flanagan family moved alcatraz island fa...
988,plot centers jim willard handsome youth virgin...
9378,hunter alien lifeform inside another resemblin...


In [24]:
test_df['description w/o SW']

12457    incident saudi arabia touches world war iii un...
1403     british risen power egypt suppress worship old...
6325     year old annabel hope jayquith beautiful famou...
11366    midsomer worthy writers circle group amateur n...
4374     book follows jason worthing also known jazz bo...
                               ...                        
4976     stephanie plum back wants revenge time shes tr...
4654     moose flanagan family moved alcatraz island fa...
988      plot centers jim willard handsome youth virgin...
9378     hunter alien lifeform inside another resemblin...
10439    narrative concern three adopted sisters paulin...
Name: description w/o SW, Length: 2569, dtype: object

In [28]:
df['description'][2810]

'in the th century humanity has spread across the galaxy first aboard hawking drive ships and then through farcasters which permit nigh instantaneous travel between them regardless of the distances the farcaster network the worldweb is the infrastructural and economical basis of the hegemony of man and thus determines the whole culture and society also flowing across these portals are the structures of the datasphere a network reminiscent of the internet in design but far more advanced in that lurks the powerful knowledgeable and utterly inscrutable technocore the vast agglomeration of millions of ais who run almost every piece of high technology of mankind the unthinking hubris of man resulted in the death of the home world earth and this arrogant philosophy was carried forth to the stars for centuries the hegemony itself is a largely decadent society relying on its military to incorporate into the worldweb the colony planets even unwillingly and also to defend the hegemony from attac

In [29]:
s = predict(df['description'][2810])

In [30]:
s

[('Non fiction', 'Science', 'Speculative')]

In [32]:
for i in s:
  #print(i)
  l2 = list(i)

l2  

('Non fiction', 'Science', 'Speculative')


['Non fiction', 'Science', 'Speculative']

In [33]:
l2[0]

'Non fiction'

In [82]:
c = 0
final_list =[]

import random 
s = predict(df['description'][2810])
for i in s:
  #print(i)
  l2 = list(i)

for i in l2:
  #j = set(i)
  ln = []
  for m in range(len(df['m-genre'])):
    if (i in df['m-genre'].iloc[m]):
      ln.append(m)
  if(len(ln)>5):
        l3 = (random.sample(ln, 5))
        print(l3)      
  for i in l3:
    #print(i)
    lgnd =[]  
    # print(df['name'].iloc[i])
    # print(df['description'].iloc[i])
    # print(df['m-genre'].iloc[i])
    lgnd.append(df['m-genre'].iloc[i]) 
    lgnd.append(df['name'].iloc[i])
    lgnd.append(df['description'].iloc[i])
    #lgnd.append(df['m-genre'].iloc[i])
    #print(lgnd)
    final_list.append(lgnd)
  
      #ld.append(df['description'].iloc[m])
      #c+=1
    # n = set(df['m-genre'].iloc[m])
    # if len(n.intersection(j)) == len(j):
    #   if len(n.intersection(j)) == len(n):
    #     print(df['name'].iloc[m])
    #     c+=1
print(final_list)
print(len(final_list))
#print(c)

[3611, 475, 5178, 8579, 6750]
[6057, 827, 2085, 11383, 12158]
[5313, 922, 6416, 932, 9117]
[[['Anthology', 'Western fiction', 'Speculative', 'Non fiction'], 'The Remnant', 'the remnant begins immediately after the end of desecration the great tribulation unfolds with one million believers gathered in petra under the protection of god as global community supreme potentate and antichrist nicolae carpathia continues to attack them and armageddon approaches carpathia is ecstatic that he is about to attack the believers with massive amounts of conventional ordnance a barrage that no one could survive without a miracle yet all the believers do not even feeling the flames that engulf their bodies a final missile hits opening a spring that drenches the fire afterwards people talking to each other hear each other in their own language chloe steele williams hannah palemoon and mac mccullum all go to greece disguised as gc officers to try to rescue george sebastian meanwhile ming toy leaves the s